# Train and Test custom YOLOv11 on Amazon SageMaker Studio

In this notebook we will train and test a custom YOLOv11 object detection CV model within Amazon SageMaker Studio. 

**Steps:**

### 0. Initial configuration.
### 1. Download a labeled dataset.
### 2. Data library in the yolov11 folder for our model to train
### 3. Train the custom YOLOv11 model. 


| ⚠️ WARNING: For this notebook to work, make sure to select the following settings in your jupyter environment: |
| -- |
Image: "PyTorch 1.10 Python 3.8 GPU Optimized"
Instance_type: "ml.g4dn.xlarge" (fast launch)

## 0. Initial Configuration (check cuda availability and run in the terminal)

In [ ]:
##export CUDA_HOME=/usr/local/cuda
##export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CUDA_HOME/lib64:$CUDA_HOME/extras/CUPTI/lib64
##export PATH=$PATH:$CUDA_HOME/bin
##source ~/.bashrc


##echo $CUDA_HOME
##echo $LD_LIBRARY_PATH
##echo $PATH

##cuda버전 확인
##nvcc --version


#### Download the YOLOv11 repository

In [ ]:
!git clone --quiet https://github.com/ultralytics/ultralytics

In [ ]:
!pip install --no-input ultralytics

In [ ]:
!pip install -U ultralytics

In [ ]:
!conda install --yes -c conda-forge opencv

In [ ]:
import os
import boto3
import glob
s3_resource = boto3.resource('s3')

## 1. Download a labeled dataset with YOLOv11 expected format.

Before we train a custom YOLOv11 model, we need to have a labeled dataset. 
In the previous notebook "0 - Label your dataset with Amazon SageMaker GroundTruth" you will be able to label your own dataset and transform it into YOLOv11 expected format or use an example custom dataset. Once you have run through one of the two options you will have available the S3 dataset location and labels used.

In [ ]:
#dataset_s3_uri = "s3://sagemaker-us-west-2-986221661979/yolov5-process-2025-01-03-11-38-21-706/output/train/training_data"
#labels = ['car number']

dataset_s3_uri = "s3://sagemaker-us-west-2-986221661979/yolov11-process-2025-01-08-13-00-11-518/output/train/training_data"
labels = ['airplane', 'car', 'ferry', 'helicopter', 'motorbike']

#### Download the dataset

In [ ]:
def split_s3_path(s3_path):
    path_parts=s3_path.replace("s3://","").split("/")
    bucket=path_parts.pop(0)
    key="/".join(path_parts)
    return bucket, key

In [ ]:
bucket,dataset_name = split_s3_path(dataset_s3_uri)
bucket,dataset_name

In [ ]:
def download_dataset(bucket_name, folder):
    bucket = s3_resource.Bucket(bucket_name) 
    for obj in bucket.objects.filter(Prefix = folder):
        if not os.path.exists(os.path.dirname(obj.key)):
            os.makedirs(os.path.dirname(obj.key))
        bucket.download_file(obj.key, obj.key)

In [ ]:
download_dataset(bucket, dataset_name)

#### Lets explore our dataset

In [ ]:
for filename in glob.iglob(dataset_name + '**/**', recursive=True):
     print(filename)

# Now let's add these data sources to the data library in the yolov11 folder for our model to train

In [ ]:
import os

current_path = os.getcwd()
print("현재 작업 디렉토리의 절대 경로:", current_path)

In [ ]:


with open("coco8.yaml", 'w') as target:
    target.write("path: {}\n".format(current_path+"/"+dataset_name))
    target.write("train: images/train\n")
    target.write("val: images/validation\n")
    target.write("names:\n")
    for i, label in enumerate(labels):
        target.write("  {}: {}\n".format(i, label))
        
with open('coco8.yaml') as file:
    lines = file.readlines()
    for line in lines:
        print(line)

# 3. Train the custom YOLOv11 model.

In [ ]:
from ultralytics import YOLO

# Load a model
#model = YOLO("yolo11n.pt")
model = YOLO("yolo11x.pt")

# Train the model
train_results = model.train(
    data="coco8.yaml",  # path to dataset YAML
    epochs=20,  # number of training epochs
    imgsz=640,  # training image size
    device="cuda",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

# Evaluate model performance on the validation set
metrics = model.val()

#model.save("best.pt")


In [ ]:
#import cv2
#import numpy as np
from PIL import Image

# Perform object detection on an image
#model = YOLO("best.pt")

image_path = './yolov11-process-2025-01-08-04-47-28-958/output/train/training_data/images/train/image_0007.jpeg'
image = Image.open(image_path)

results = model(image, conf=0.25, imgsz=640)
#results = model("./validation_dataset/bike.jpg", conf=0.25, imgsz=640)
results[0].show()

# 추론 결과 출력
print(results)

# Export the model to ONNX format
#path = model.export(format="onnx")  # return path to exported model
#print("export model path:", path)



### 4. Make inferences with the created model.

In [ ]:
#!python yolov5/detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.5 --source ""

| ⚠️ WARNING: Remember to shutdown the instance once finalized with this notebook to prevent unnecesary charges. Head to running Terminals and Kernels tab and shutdown the running instance. |
| -- |